In [9]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

from pymongo import MongoClient
from bson.objectid import ObjectId

import time,datetime
import os 
import schedule

import pandas as pd
import pandas_datareader.data as web
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import style

from keras.models import Model, Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import keras

from sklearn import preprocessing

from copy import deepcopy
from IPython.display import clear_output

In [2]:
style.use('ggplot')

In [5]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences[0])):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences[0])-1:
            break
        # gather input and output parts of the pattern
        seq_x = sequences[0][i:end_ix].reshape(len(sequences[0][i:end_ix]),1) 
        seq_x = np.append(seq_x, sequences[1][i:end_ix].reshape(len(sequences[1][i:end_ix]),1), axis=1)
        seq_x = np.append(seq_x, sequences[2][i:end_ix].reshape(len(sequences[2][i:end_ix]),1), axis=1)
        seq_y1 =  sequences[0][end_ix]
        X.append(seq_x)
        y.append(seq_y1)
    return np.array(X), np.array(y)

In [6]:
def start_prediction():
    conn = MongoClient('120.126.136.17')
    db = conn.Tracker
    collection = [db.james]# db.db2, db.dn2, db.james, db.leo
    clean_data = []
    for col in collection:
        cursor = col.find({})
        df = pd.DataFrame(list(cursor))

    df.replace('', np.nan, inplace=True)
    df.fillna(method='ffill', inplace=True)

    df['hr_value'] = df['hr_value'].astype(float)
    df['o2_value'] = df['o2_value'].astype(float)
    df['latitude'] = df['latitude'].astype(float)
    df['longitude'] = df['longitude'].astype(float)
    df['step_value'] = df['step_value'].astype(float)

    df = df[(df['hr_value'] != 0)]
    # map out the weekday
    df['weekday'] = df['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%w'))
    df['week'] = df['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x).isocalendar()[1]
              - (datetime.datetime.fromtimestamp(x).isoweekday() < 7)) # do this so that week may start on sunday

    dfsteptotal = pd.DataFrame()
    dfsteptotal = df[['week', 'weekday']].set_index(['week', 'weekday'])
    dfsteptotal = dfsteptotal[~dfsteptotal.index.duplicated(keep='first')]
    dfsteptotal['steps'] = 0
    dfsteptotal.reset_index(inplace=True)
    dfadd = pd.DataFrame({'week' : [12], 'weekday':['5'], 'steps':[0]})
    dfsteptotal = dfsteptotal.append(dfadd, ignore_index=True)
    dfsteptotal = dfsteptotal.set_index(['week', 'weekday'])
    dfsteptotal.sort_index(inplace=True)

    # here I map between integer and the sting it's mean
    timeslidemap = {0:'morning', 1:'afternoon', 2:'evening'}
    basestamp = df['timestamp'][0]
    endstamp = df['timestamp'][-1:].values[0]
    # for num, row in df[:30000].iterrows():
    while True:
        try:
            dfbetween = df.loc[df['timestamp'].between(
                datetime.datetime.fromtimestamp(basestamp).replace(hour=0,minute=0,second=0).timestamp(),
                datetime.datetime.fromtimestamp(basestamp).replace(hour=23,minute=59,second=59).timestamp()
                        , inclusive=True)]
            steps = dfbetween['step_value'].sum()
        #     print(int(datetime.datetime.fromtimestamp(basestamp).strftime('%m')),
        #             int(datetime.datetime.fromtimestamp(basestamp).strftime('%d')),
        #                 timeslidemap[estimedevide])

            dfsteptotal.loc[[(datetime.datetime.fromtimestamp(basestamp).isocalendar()[1] - 
                                        (datetime.datetime.fromtimestamp(basestamp).isoweekday() < 7),
                                datetime.datetime.fromtimestamp(basestamp).strftime('%w'))], 
                                               ['steps']] += steps

            basestamp = (datetime.datetime.fromtimestamp(basestamp) + datetime.timedelta(days=1)).timestamp()
            if basestamp > endstamp:
                break
        except Exception as e:
            print(str(e))
            basestamp = (datetime.datetime.fromtimestamp(basestamp) + datetime.timedelta(days=1)).timestamp()
    #     else:
    #         print(datetime.datetime.fromtimestamp(basestamp) ,datetime.datetime.fromtimestamp(endstamp))


    dfsteptotal.loc[(12,'5')] = dfsteptotal.xs('5', level=1).sum() / (dfsteptotal.xs('5', level=1).count() - 1)
    dfsteptotal['Comfort'] = 0
    dfsteptotal['Precp'] = 0

    # add wether 
    coding = 'utf-8'
    table = pd.DataFrame()
    basetime = datetime.date(2019,3,27)
    num = 0
    for index, row in dfsteptotal.iterrows():
        try:
            # parsing part
            date = basetime + datetime.timedelta(days = num)
            num += 1
            dateStr = date.strftime('%Y-%m-%d')
            url = 'https://e-service.cwb.gov.tw/HistoryDataQuery/DayDataController.do?command=viewMain&station=C0AC60'+ \
                '&stname=%25E4%25B8%2589%25E5%25B3%25BD&datepicker=' + dateStr
            table = pd.read_html(url,encoding=coding,index_col=0,header=None,flavor='bs4')[1]
            # start dataprocess
            table.replace('/', np.nan, inplace=True)
            table.replace('X', np.nan, inplace=True)  
            table.fillna(method='ffill', inplace=True)
            table.index -= 1
            dfoneday = pd.DataFrame()
            # the table data need to reshape and turn to a variable
            T = table['temperature'].values.reshape(len(table['temperature'].values)).astype('float')
            RH = table['RH'].values.reshape(len(table['RH'].values)).astype('float')
            dfoneday['comfort'] = T - 0.55 *(1-RH/100)*(T - 14)
            dfoneday['Precp'] = table['Precp']['降水量(mm)']
            bins = [0,11,16,20,27,31,60]
            labels=[0,1,2,3,4,5]
            dfoneday['bins'] = pd.cut(dfoneday['comfort'], bins=bins, labels=labels, include_lowest=True)
            # mComfort aComfort eComfort mPrecp aPrecp ePrecp
            dfsteptotal.loc[[index], ['Comfort']] = dfoneday['bins'].astype(int).mean()
            dfoneday['Precp'].astype(float)
            dfsteptotal.loc[[index], ['Precp']] = dfoneday['Precp'].astype(float).sum()
        except Exception as e:
            if str(e) == "could not convert string to float: '...'":
                print('some data are not yet ready')
                break
    print(date)

    steps = dfsteptotal['steps'][:-1].values
    Comfort = dfsteptotal['Comfort'][:-1].values
    Precp = dfsteptotal['Precp'][:-1].values

    scaler = preprocessing.StandardScaler()
    dataset = np.array([steps, Comfort, Precp])

    scaler = preprocessing.StandardScaler()
    dataset = np.array([steps, Comfort, Precp])
    normalized_dataset = scaler.fit_transform(dataset)
    n_steps = 3
    X, y = split_sequences(normalized_dataset, n_steps)

    n_features = 3
    model = Sequential()
    model.add(keras.layers.LSTM(100,activation='relu', return_sequences=True,input_shape=(n_steps, n_features)))
    model.add(keras.layers.LSTM(100, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')


    history = model.fit(X, y, epochs=50, verbose=0, batch_size=128, validation_split=0.2)

    newdataset = dataset[:, -3:]
    normalized_dataset = scaler.fit_transform(newdataset.reshape(-1,1))
    normalized_dataset = normalized_dataset.reshape(1,3,3)

    p = model.predict(normalized_dataset)[0]
    prediction = int(scaler.inverse_transform(p))
    date = basetime + datetime.timedelta(days = num)
    dateStr = date.strftime('%Y-%m-%d')
    lis = dateStr.split('-')
    lis = list(map(int, lis))

    conn = MongoClient('120.126.136.17')
    db = conn.Tracker
    collection = db.prediction# db.db2, db.dn2, db.james, db.leo
    mydict = { 'year':lis[0], 'month':lis[1], 'day':lis[2], 'prediction':prediction}
    collection.insert_one(mydict)

In [18]:
schedule.every().day.at("1:30").do(start_prediction)

ScheduleValueError: Invalid time format

In [16]:
while True:
    schedule.run_pending()

some data are not yet ready
2019-04-25
some data are not yet ready
2019-04-25
some data are not yet ready
2019-04-25
some data are not yet ready
2019-04-25
some data are not yet ready
2019-04-25
some data are not yet ready
2019-04-25
some data are not yet ready
2019-04-25
some data are not yet ready
2019-04-25
some data are not yet ready
2019-04-25


KeyboardInterrupt: 